In [86]:
import numpy as np
import glob
import os
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices(device_type='GPU') #本地需要这样操作
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu ,True)

In [87]:
x_train_path=glob.glob('./dataset/dc_2000/train/*/*.jpg')
x_test_path=glob.glob('./dataset/dc_2000/test/*/*.jpg')

In [88]:
len(x_train_path)

2000

In [89]:
#cat 0
#dog 1
y_train=[int(p.split('\\')[1]=='dog') for p in x_train_path]
y_test=[int(p.split('\\')[1]=='dog') for p in x_test_path]

In [3]:
def load_preprosess_image(img_path):
    img_raw=tf.io.read_file(img_path)   
    img_tensor=tf.image.decode_jpeg(img_raw,channels=3)
    #普通方法
	#img_tensor=tf.image.resize(img_tensor,[256,256])
    #数据增强
    img_tensor=tf.image.resize(img_tensor,[360,360])
    img_tensor=tf.image.random_crop(img_tensor,[256,256,3])#随机裁剪
    img_tensor=tf.image.random_flip_left_right(img_tensor) #随机左右翻转
    img_tensor=tf.image.random_flip_up_down(img_tensor) #随机上下翻转
    #下面的效果不明显
    #img_tensor=tf.image.random_brightness(img_tensor,0.5) #随机改变亮度
    #img_tensor=tf.image.random_contrast(img_tensor,0,1) #随机对比度
    #img_tensor=tf.image.random_hue(img_tensor,0.3) #随机颜色
    #img_tensor=tf.image.random_saturation(img_tensor,0.2,1.8)#随机饱和度
    ##
    
    img_tensor=tf.cast(img_tensor,tf.float32)
    img_tensor=img_tensor/255
    return img_tensor

In [91]:
def load_preprosess_image_test(img_path):
    img_raw=tf.io.read_file(img_path)   
    img_tensor=tf.image.decode_jpeg(img_raw,channels=3)
    img_tensor=tf.image.resize(img_tensor,[256,256])
    img_tensor=tf.cast(img_tensor,tf.float32)
    img_tensor=img_tensor/255
    return img_tensor

In [ ]:
#label : [1, 2, 3]==>[ [1], [2], [3]]
y_train=tf.reshape(y_train,[-1,1]) 

In [92]:
#自动并行运行
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [93]:
train_path_dataset=tf.data.Dataset.from_tensor_slices(x_train_path)
x_train_dataset=train_path_dataset.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)
y_train_dataset=tf.data.Dataset.from_tensor_slices(y_train)
dataset_train=tf.data.Dataset.zip((x_train_dataset,y_train_dataset))

In [94]:
test_path_dataset=tf.data.Dataset.from_tensor_slices(x_test_path)
x_test_dataset=test_path_dataset.map(load_preprosess_image_test,num_parallel_calls=AUTOTUNE)
y_test_dataset=tf.data.Dataset.from_tensor_slices(y_test)
dataset_test=tf.data.Dataset.zip((x_test_dataset,y_test_dataset))

In [95]:
BATCH_SIZE=32
train_count=len(x_train_path)
dataset_train=dataset_train.shuffle(train_count).batch(BATCH_SIZE).prefetch(AUTOTUNE)
#prefetch是预加载
dataset_test=dataset_test.batch(BATCH_SIZE).prefetch(AUTOTUNE)



In [ ]:
model=tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),activation='relu'),
        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
        tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
        tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
        tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ]
)



In [97]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 125, 125, 128)     73856     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 60, 60, 256)       295168    
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 28, 28, 512)      

In [98]:
x,y=next(iter(dataset_train))

In [99]:
pred=model.predict(x)

In [100]:
tf.reshape(tf.cast(pred>0,tf.int32),[-1])

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>

In [101]:
tf.reshape(y,[-1])



<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0])>

In [102]:
optimizer=tf.keras.optimizers.Adam()

In [103]:
epoch_loss_avg_train=tf.keras.metrics.Mean('train_loss')
train_accuracy=tf.keras.metrics.Accuracy()
epoch_loss_avg_test=tf.keras.metrics.Mean('test_loss')
test_accuracy=tf.keras.metrics.Accuracy()

In [ ]:
#自定义训练保存模型
##cp_dir='./customtrain_cp'
##cp_prefix='./customtrain_cp/ckpt'
##checkpoint=tf.train.Checkpoint(optimizer=optimizer, model=model)


In [104]:
def train_step(model,x_train,y_train):
    with tf.GradientTape() as t:
        y_pred=model(x_train) #写成model.predict就不可以训练了
        loss_step=tf.keras.losses.BinaryCrossentropy()(y_train,y_pred)
    
    grads=t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    epoch_loss_avg_train(loss_step)
    train_accuracy(y_train,tf.cast(y_pred>0.5,tf.int32))

In [105]:
train_loss_res=[]
train_acc_loss=[]
test_loss_res=[]
test_acc_loss=[]

In [106]:
def test_step(model,x_test,y_test):
    y_pred=model.predict(x_test) #model(x_test,training=False)
    loss_step=tf.keras.losses.BinaryCrossentropy()(y_test,y_pred)
    epoch_loss_avg_test(loss_step)
    test_accuracy(y_test,tf.cast(y_pred>0.5,tf.int32))

In [107]:
epochs=30

In [108]:
def train():
    for epoch in range(epochs):
        for x,y in dataset_train:
            train_step(model,x,y)
            print('.',end='')
        print()
        train_loss_res.append(epoch_loss_avg_train.result())
        train_acc_loss.append(train_accuracy.result())
        
        for x,y in dataset_test:
            test_step(model,x,y)
        train_loss_res.append(epoch_loss_avg_test.result())
        train_acc_loss.append(test_accuracy.result())
        template = 'Epoch{}, loss: {:.3f}, acc: {:.3f} , test_loss: {}, test_acc: {}'
        print(
            template.format(
                epoch+1,
                epoch_loss_avg_train.result(),
                train_accuracy.result(),
                epoch_loss_avg_test.result(),
                test_accuracy.result(),
            )
        )
        epoch_loss_avg_train.reset_states()
        train_accuracy.reset_states()
        epoch_loss_avg_test.reset_states()
        test_accuracy.reset_states()
        #保存检查点 ，每两步保存一次 
        ##if(epoch+1)%2==0:
        ##   checkpoint.save(file_prefix=cp_prefix )

In [109]:
train()



...............................................................
Epoch1, loss: 0.694, acc: 0.493 , test_loss: 0.6895403861999512, test_acc: 0.5
...............................................................
Epoch2, loss: 0.694, acc: 0.506 , test_loss: 0.6935567855834961, test_acc: 0.5


In [109]:
#恢复检查点
##checkpoint.restore(tf.train.latest_checkpoint(cp_dir))
#